In [5]:
%%bash
pip install terra-notebook-utils --user

In [7]:
#Restart the kernal after running the above command, then continue from the cell below.

In [21]:
import os
import requests
from terra_notebook_utils import azure_auth

In [42]:
DOMAIN = 'dsde-prod.broadinstitute.org'
LEO_BASE_URL = f'https://leonardo.{DOMAIN}/api/apps/v2'
WORKSPACE_ID = os.environ['WORKSPACE_ID']

def add_to_beginning(s, start='id_'):
    return start + str(s)

def get_token():
    """Get Azure access token"""
    #Get this string from devtools -> disks?role... -> Headers -> Authorization: Bearer
    return "eyJhbGciOiJSUzI1NiIsImtpZCI6IlotT2t3M2Y2bmloQjJkaHJ4Y3ZvNEVGM2pLVkp6REZQamJ3VWJOcFZlbzAiLCJ0eXAiOiJKV1QifQ.eyJlbWFpbCI6Im1zaGFuZEBicm9hZGluc3RpdHV0ZS5vcmciLCJnaXZlbl9uYW1lIjoiTWVnYW4iLCJmYW1pbHlfbmFtZSI6IlNoYW5kIiwibmFtZSI6Ik1lZ2FuIFNoYW5kIiwiaWRwIjoiZ29vZ2xlLmNvbSIsImlkcF9hY2Nlc3NfdG9rZW4iOiJ5YTI5LmEwQWZCX2J5RGVtSTBZZjl5RG9ib0o4TUItYXJ5VlA2dEtKMWhTdFBPbkV2Vnluc0xHclJILWhzcE42YUNBejV5bXRpemJ4bkpxYkw1YnQxbm1kQUdpSVdTM3hmX1JxbUx6TF9tNjFPR2IzLS00a25mdW5qcVEwbE9kZzJXTF9GS0w0eVRpVHhJcnI1T0VUQTREanBwSVYtaDk5UFZEVnN5ZFZmTlA3SkJSUXlOcGFDZ1lLQVZZU0FSRVNGUUhHWDJNaTJsQ05rblpxTE1fN19qT1Jsei1qeUEwMTc5IiwiZ29vZ2xlX2lkIjoiMTE3OTUxMjU2Nzc4MTcxNDg5MTg1IiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FDZzhvY0xnOGhza2pKa0pETjJZTU4xMnpyZ0tfdEJIcTNUOF9uQS1nV1dTVS01ST1zOTYtYyIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJzdWIiOiI5NWI5M2M5YS02MDZiLTRhNjktYjk2Ni0yYmJjYzFmMmY2NGMiLCJ0aWQiOiIzNWVhNWRlOS0xNzg2LTRiYmItODljZi0yNWI4ODI2MTY0M2QiLCJhenAiOiI0YWVmYzRhYy1kNWJlLTQ0NDAtYWU5YS1hZWZlNGNlYjA5OGMiLCJ2ZXIiOiIxLjAiLCJpYXQiOjE3MDg5ODE0MDUsImF1ZCI6IjRhZWZjNGFjLWQ1YmUtNDQ0MC1hZTlhLWFlZmU0Y2ViMDk4YyIsImV4cCI6MTcwODk4NTAwNSwiaXNzIjoiaHR0cHM6Ly90ZXJyYXByb2RiMmMuYjJjbG9naW4uY29tLzM1ZWE1ZGU5LTE3ODYtNGJiYi04OWNmLTI1Yjg4MjYxNjQzZC92Mi4wLyIsIm5iZiI6MTcwODk4MTQwNX0.mQSB7iLROI75H_L6RiloXqL7zr_3ivcYjjbvPfeniSjmnThslFLYLTnm3m9B7mAFJcNdP8gDQpTyq8SnaPSKtJsA795xvwUvpugMKHsXmo2uowrA6s-Cm6L82_WjU-frg36kZuxiStXVelhrTGRzZSPv7V6R-VBD-4F0IwmnFh7m6OfrV7vDnkz2hA6tBgDxTa84Rx7Vi0ce1uKDgRO7oVYbMUxh_I4tShY1mh1l-nBFCB6QXQcZaO50MIJGYos4R7OVq2AkQhsFRN8b70_Nx6nV9hMUOUfDim_ax0l7g8-3YhOihECXDOTzONVr7PYaoDBTmd6NuIMh0BQCu7X82w"

def headers():
    return {
        "Authorization": f"Bearer {get_token()}",
        "Accept": "application/json"
    }

def get_cbas_url():
    """Get url for CBAS"""
    uri = f"{LEO_BASE_URL}/{WORKSPACE_ID}?includeDeleted=false"
    response = requests.get(uri, headers=headers())
    status_code = response.status_code
    if status_code != 200:
        return response.text
    response_json = response.json()
    for item in response_json:
        if item['appType'] == 'WORKFLOWS_APP':
            return item['proxyUrls']['cbas']
    return json.loads(response.text)['proxyUrls']['wds']

def get_post_data(start, end):
    return {
  "run_set_name": f"SingleSample_{start}-{end}",
  "run_set_description": "",
  "method_version_id": "bd5f599d-e7df-44d1-9488-5fcc9b59e72b",
  "workflow_input_definitions": [
    {
      "input_name": "SingleSamplePipeline.name",
      "input_type": {
        "type": "primitive",
        "primitive_type": "String"
      },
      "source": {
        "type": "record_lookup",
        "record_attribute": "test_type_id"
      }
    },
    {
      "input_name": "SingleSamplePipeline.value",
      "input_type": {
        "type": "primitive",
        "primitive_type": "Int"
      },
      "source": {
        "type": "record_lookup",
        "record_attribute": "input_value"
      }
    }
  ],
  "workflow_output_definitions": [
    {
      "output_name": "SingleSamplePipeline.out",
      "output_type": {
        "type": "primitive",
        "primitive_type": "File"
      },
      "destination": {
        "type": "record_update",
        "record_attribute": "out_file"
      }
    }
  ],
  "wds_records": {
    "record_type": "test_type",
    "record_ids": list(map(add_to_beginning, range(start,end))),
  },
  "call_caching_enabled": True
}

def submit_to_cbas(cbas_url, body):
    uri = f"{cbas_url}/api/batch/v1/run_sets"
    response = requests.post(uri, json=body, headers=headers())
    status_code = response.status_code
    if status_code != 200:
        raise Exception(response.text)
    return response.json()
    

In [43]:
batch = 10
start = 10
end = 100

In [44]:
cbas_url = get_cbas_url()
i = start
while i < end:
    print(f"Submitting batch {i}..{i+batch-1}")
    data = get_post_data(i, i+batch)
    submit_to_cbas(cbas_url, data)
    i = i + batch
print("Done")
    

Submitting batch 10..19
Submitting batch 20..29
Submitting batch 30..39
Submitting batch 40..49
Submitting batch 50..59
Submitting batch 60..69
Submitting batch 70..79
Submitting batch 80..89
Submitting batch 90..99
Done
